    # ant control

In [1]:
from graph_tool.all import *
import numpy as np
import uuid
from ant import Ant
from collections import Counter
from time import sleep

np.random.seed(43)
seed_rng(42)

# We need some Gtk and gobject functions
from gi.repository import Gtk, Gdk, GdkPixbuf, GObject

'''


'''


# Generate random graph for simulation
'''
max_deg = 5
def sample_k(max):
    accept = False
    while not accept:
        k = np.random.randint(1,max+1)
        accept = np.random.random() < 1.0/k
    return k

g = random_graph(100, lambda: sample_k(max_deg), model="probabilistic",
                    vertex_corr=lambda i, k: 1.0 / (1 + abs(i - k)), directed=False,
                    n_iter=100)

pos = sfdp_layout(g)

g = collection.data["netscience"]
g = GraphView(g, vfilt=label_largest_component(g), directed=False)
g = Graph(g, prune=True)

# Complete graph
g = complete_graph(10)
pos = sfdp_layout(g, cooling_step=0.95, epsilon=1e-2)
'''

g, pos = triangulation(np.random.random((30, 2)) * 4, type="delaunay")
#g, pos = geometric_graph(np.random.random((1000, 2)) * 4, radius=.3,)

#g = lattice([15,15])
#pos = sfdp_layout(g, cooling_step=0.95, epsilon=1e-2)

#tree = min_spanning_tree(g)
#g = GraphView(g, vfilt=label_largest_component(g), directed=False)
#graph_draw(g, pos=pos, edge_color=tree)

# We use directly the color of the vertices in (R,G,B,A) format.

W = [1, 1, 1, 1]           # White color
B = [0, 0, 0, 1]           # Black color


# Define properties
state = g.new_vertex_property("vector<double>")
weight = g.new_edge_property("double")
pheromone = g.new_edge_property("double")
pheromone_concentration = g.new_edge_property("double")
weight_str = g.new_edge_property("string")

removed = g.new_edge_property('bool')

# Iternalize properties
g.ep.weight = weight
g.ep.pheromone_concentration = pheromone_concentration
g.ep.pheromone = pheromone

# Initialize graph properties
# Vertex properties
state.set_value(W)

# edge properties

pheromone.set_value(1.)
pheromone_concentration.set_value(1.5)
weight_str.set_value(' ')

weight.a = np.random.randint(2,20,len(weight.a)) * 1.

# Translate weights for later display

for e in g.edges():
    weight_str[e] = str(weight[e])
'''    '''
    
# Get simulation data from graph

# Get nest node

_, selected = graph_draw(g, pos=pos , edge_text=weight_str, vertex_text=g.vertex_index)

nest_node_idx = np.where(selected.a==1)[0]

# Get food node

_, selected = graph_draw(g, pos=pos  , edge_text=weight_str, vertex_text=g.vertex_index)

food_node_idx = np.where(selected.a==1)[0]

# This creates a GTK+ window with the initial graph layout
win = GraphWindow(g, pos, geometry=(600, 500),
                      edge_color=B,
                      edge_pen_width=pheromone,
                      edge_text = weight_str,
                      vertex_fill_color=state,
                      #vertex_text=g.vertex_index,
                      vertex_halo_color=[0.8, 0, 0, 0.6])

# Agents count
agents_count = 30

print 'source ', nest_node_idx, ' nodes to visit ',food_node_idx

# Spawn agents 
# Ant simulation
agents = [ Ant(g, start_nodes=nest_node_idx, uid=uuid.uuid4(), alpha=1., beta=1., ntv=food_node_idx) for agent in np.arange(agents_count)]

# General TSP Solver - initialization

cnodes = [ agent.get_cn() for agent in agents ]

# Parameter to control pheromone evaporation

rho = .85

# Iteration tracking

max_iteration = 90
i_counter = 0

def update_state():
    global agents_count, i_counter, agents, max_iteration, rho, nest_node, food_node

    # Reset vertex state
    state.set_value(W)
    
    # Get active agents
    aa = [ agent for agent in agents if agent.isActive()]
    
    if aa:        
        nnodes = [ agent.move() for agent in aa ]
        for v in nnodes:
            state[v] = B
    
        # Nest and Food
        if (len(nest_node_idx) < 2) and (len(food_node_idx) <2):
            state[g.vertex(nest_node_idx)] = G
            state[g.vertex(food_node_idx)] = G
        
    
    # If all agents are done do the bookkeeping and prepare graph for another epoch
    else:
        # Take only info if agent is not MIA
        # Round agents with actual solutions
        aa = [ agent for agent in agents if not agent.isMIA() ]

        # Get solutions
        agent_solutions = [ agent.get_path() for agent in aa]
        
        # Get solution costs
        agent_solution_cost = [ agent.dump_path_cost() for agent in aa ]
        
        # Get individual pheromone trails and consolidate for global update 
        agent_trail = [ agent.dump_trail() for agent in aa ]
        
        delta_trail = Counter({})
        
        for trail in agent_trail:
            delta_trail = delta_trail + Counter(trail)
        
        # Evaporate pheromones
        
        pheromone_concentration.a = (1 - rho) * np.array(pheromone_concentration.a)
        
        for e in delta_trail.keys():
            try:
                pheromone_concentration[e] = pheromone_concentration[e] + delta_trail[e]
            except:
                pass
                   
        pheromone.a = 3. * np.array(pheromone_concentration.a)
        
        '''
        # OPTIONAL - Update edge info - show label only for some edges
        
        for e in g.edges():
            if pheromone_concentration[e] > .85:
                weight_str[e] = str(weight[e])
            else:
                weight_str[e] = ''
        '''
        
        if i_counter < max_iteration:
            
            # Reset agents
                        
            for agent in agents:
                agent.reset()
                
            # Keep track of epochs
            i_counter += 1
        else:
            best_path_cost, best_path_id = np.min(agent_solution_cost), agent_solutions[np.argmin(agent_solution_cost)]
            print best_path_cost, best_path_id
            
            return False   
    
    # The following will force the re-drawing of the graph, and issue a
    # re-drawing of the GTK window.
    win.graph.regenerate_surface()
    win.graph.queue_draw()
    
    # We need to return True so that the main loop will call this function more
    # than once.
    return True

# Bind the function above as an 'idle' callback.
cid = GObject.idle_add(update_state)

# We will give the user the ability to stop the program by closing the window.
win.connect("delete_event", Gtk.main_quit)

# Actually show the window, and start the main loop.
win.show_all()
Gtk.main()

source  [4]  nodes to visit  [29]


/home/pc/.local/lib/python2.7/site-packages/ipykernel/__main__.py:233: PyGIDeprecationWarning: GObject.idle_add is deprecated; use GLib.idle_add instead


12.0 [4, 0, 16, 29]


In [ ]:
print best_path_cost, best_path_id
graph_draw(g, pos=pos , edge_text=weight_str, vertex_text=g.vertex_index)